**Задание 1**
Напишите функцию, которая классифицирует фильмы из материалов занятия по следующим правилам:
  - оценка 2 и меньше - низкий рейтинг
  - оценка 4 и меньше - средний рейтинг
  - оценка 4.5 и 5 - высокий рейтинг

Результат классификации запишите в столбец class


In [1]:
import pandas as pd

In [2]:
movies = pd.read_csv('movies.csv')
ratings = pd.read_csv('ratings.csv')

In [3]:
movies_ratings = movies.merge(ratings, on='movieId', how='left')

In [4]:
movies_ratings.head()

,movieId,title,genres,userId,rating,timestamp
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,1.0,4.0,9.649827e+08
1,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,5.0,4.0,8.474350e+08
2,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,7.0,4.5,1.106636e+09
3,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,15.0,2.5,1.510578e+09
4,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,17.0,4.5,1.305696e+09


In [5]:
def class_rating(rating):
    if rating <= 2.0:
        return 'низкий рейтинг'
    elif rating <= 4.0:
        return 'средний рейтинг'
    elif (rating == 4.5) | (rating == 5.0):
        return 'высокий рейтинг'

In [6]:
# присваеваем каждой записи в дата фрейме класс рейтинга
movies_ratings['class'] = movies_ratings['rating'].apply(class_rating)
movies_ratings.head()[['title', 'rating', 'class']]

,title,rating,class
0,Toy Story (1995),4.0,средний рейтинг
1,Toy Story (1995),4.0,средний рейтинг
2,Toy Story (1995),4.5,высокий рейтинг
3,Toy Story (1995),2.5,средний рейтинг
4,Toy Story (1995),4.5,высокий рейтинг


In [7]:
# считаем колличество оценок по категориям
movies_ratings.groupby('class').count()[['title']].sort_values('title', ascending=False).head()

,title
class,
средний рейтинг,65551
высокий рейтинг,21762
низкий рейтинг,13523


In [8]:
# считаем колличество оценок по категориям
movies_ratings['class'].value_counts()

средний рейтинг    65551
высокий рейтинг    21762
низкий рейтинг     13523
Name: class, dtype: int64

**Задание 2**
Используем файл keywords.csv.

Необходимо написать гео-классификатор, который каждой строке сможет выставить географическую принадлежность определенному региону. Т. е. если поисковый запрос содержит название города региона, то в столбце 'region' пишется название этого региона. Если поисковый запрос не содержит названия города, то ставим 'undefined'.

Правила распределения по регионам Центр, Северо-Запад и Дальний Восток:

geo_data = {

    'Центр': ['москва', 'тула', 'ярославль'],

    'Северо-Запад': ['петербург', 'псков', 'мурманск'],

    'Дальний Восток': ['владивосток', 'сахалин', 'хабаровск']

}

Результат классификации запишите в отдельный столбец region.



In [9]:
df = pd.read_csv('keywords.csv')

In [10]:
geo_data = {

'Центр': ['москва', 'тула', 'ярославль'],

'Северо-Запад': ['петербург', 'псков', 'мурманск'],

'Дальний Восток': ['владивосток', 'сахалин', 'хабаровск']
}

In [11]:
def geo_classificator(x):
    for key, list_ in geo_data.items():
        for town in list_:
            if town in x:
                return key
    return 'undefined'

In [12]:
%%time
df['region'] = df['keyword'].apply(geo_classificator)
df.head()

Wall time: 107 ms


,keyword,shows,region
0,вк,64292779,undefined
1,одноклассники,63810309,undefined
2,порно,41747114,undefined
3,ютуб,39995567,undefined
4,вконтакте,21014195,undefined


In [13]:
df.region.value_counts()

undefined         99253
Центр               387
Северо-Запад        266
Дальний Восток       94
Name: region, dtype: int64

**Задание 3 (бонусное)**
Есть мнение, что "раньше снимали настоящее кино, не то что сейчас". Ваша задача проверить это утверждение, используя файлы с рейтингами фильмов из прошлого домашнего занятия (файл ratings.csv из базы https://grouplens.org/datasets/movielens). Т. е. проверить верно ли, что с ростом года выпуска фильма его средний рейтинг становится ниже.

При этом мы не будем затрагивать субьективные факторы выставления этих рейтингов, а пройдемся по следующему алгоритму:

1. В переменную years запишите список из всех годов с 1950 по 2010.

2. Напишите функцию production_year, которая каждой строке из названия фильма выставляет год выпуска. Не все названия фильмов содержат год выпуска в одинаковом формате, поэтому используйте следующий алгоритм:
  - для каждой строки пройдите по всем годам списка years
  - если номер года присутствует в названии фильма, то функция возвращает этот год как год выпуска
  - если ни один из номеров года списка years не встретился в названии фильма, то возвращается 1900 год


3. Запишите год выпуска фильма по алгоритму пункта 2 в новый столбец 'year'

4. Посчитайте средний рейтинг всех фильмов для каждого значения столбца 'year' и отсортируйте результат по убыванию рейтинга


In [36]:
years = list(range(1950, 2011))

In [28]:
ratings = pd.read_csv('ratings.csv')
movies = pd.read_csv('movies.csv')
joined = ratings.merge(movies, on='movieId', how='left')

In [30]:
def record_of_year(x):
    for year in years:
        if str(year) in x:
            return str(year)
    return '1900'

In [31]:
joined['year'] = joined['title'].apply(record_of_year)
joined.head()

,userId,movieId,rating,timestamp,title,genres,year
0,1,1,4.0,964982703,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,1995
1,1,3,4.0,964981247,Grumpier Old Men (1995),Comedy|Romance,1995
2,1,6,4.0,964982224,Heat (1995),Action|Crime|Thriller,1995
3,1,47,5.0,964983815,Seven (a.k.a. Se7en) (1995),Mystery|Thriller,1995
4,1,50,5.0,964982931,"Usual Suspects, The (1995)",Crime|Mystery|Thriller,1995


In [44]:
joined.groupby('year').mean()[['rating']].sort_values('rating', ascending=False).head(10)

,rating
year,
1957,4.039535
1954,4.009191
1962,3.969466
1952,3.953125
1972,3.944293
1964,3.940160
1974,3.935622
1967,3.922572
1975,3.879121


**Вывод**  Корреляция между увеличением года выпуска фильма и его средним рейтингом не высокая.